# Random Forest
Da die lineare Regression aufgrund der Ungleichverteilung der Auslastung nicht gut funktioniert hat, wurden andere Möglichkeiten evaluiert. Eines der bekanntesten Machine Learning Verfahren, ist der Random Forest, welcher sehr gut beide, also Lineare- und Klassen-Zusammenhänge, darstellen und hervorsagen kann. Random Forest basiert auf Entscheidungsbäume. Bei jeder Verästelung im Entscheidungsbaum wird ein Feature benutzt um festlegen zu können, in welche Klasse der Entscheid fällt. Sprich in unserem Beispiel haben wir die freien Parkplätze pro Stunde berechnet, und diese dann auf Testdaten angewendet. Das Ergebnis war viel zufriedenstellender, als bei der Linearen Regression. Mehr dazu später.

In [4]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_pickle("./data.pkl")

In [5]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

for parkingid in df['parking_id'].unique():
    data = df[df['parking_id'] == parkingid]
    X = data[['day_of_month', 'day_of_week', 'month', 'is_weekend', 'daypart_cat']]
    y = data['free_parking_0']
    
    # Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2) # 80% training and 20% test
    print(y_train.shape)
    forest = RandomForestRegressor(n_estimators=9)
    forest.fit(X_train, y_train)
    score = forest.score(X_test, y_test)
    print(score)
    first_tree = forest.estimators_[0]
    feat_imp = pd.Series(forest.feature_importances_, index=X.columns)
    print(feat_imp.sort_values(ascending=False))

(64385,)
0.878450317003769
day_of_month    0.494310
month           0.273862
day_of_week     0.137628
daypart_cat     0.083305
is_weekend      0.010895
dtype: float64
(64385,)
0.7949703434889533
day_of_month    0.341077
daypart_cat     0.271235
day_of_week     0.185445
month           0.132971
is_weekend      0.069272
dtype: float64
(64385,)
0.8646179249749109
daypart_cat     0.478215
is_weekend      0.234238
month           0.135754
day_of_month    0.093476
day_of_week     0.058317
dtype: float64
(64385,)
0.8429506285395159
daypart_cat     0.445558
month           0.211331
day_of_month    0.189665
day_of_week     0.110596
is_weekend      0.042850
dtype: float64
(64385,)
0.8797569144734909
daypart_cat     0.427729
month           0.273254
day_of_month    0.171082
day_of_week     0.099260
is_weekend      0.028675
dtype: float64
(64385,)
0.8354755059263571
month           0.314762
daypart_cat     0.259984
is_weekend      0.196703
day_of_month    0.184258
day_of_week     0.044294
dtype: f

# Resultat
Das Projektteam war begeistert der guten Resultate welche die Modelle erziehlten. Leider wurde realisiert, dass das etwas nicht stimmen konnte. Die Performance war viel zu gut. Nach einigen Tagen studieren kam dir erleuchtung. Der Train Test Split war nucht gut gewählt, denn dadurch dass die Daten nach dem 80 - 20 Prinzip aufgeteil wurden hatte das Modell leichtes spiel, da es die erwarteten Daten bis auf fünf Minuten bereis gesehen hat. Also wird gefragt wie frei war das Parkhaus am 28.10.2020 um 18:05 hat das Model sehr wahrscheindlich die Daten für 18:00 und 18:10 bereits gesehen, was bedeutet die Werte sind nicht weit davon entfernt. 